In [1]:
import pandas as pd
data = pd.read_csv('training_data.csv')

In [2]:
data.describe()

,Facies,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
count,3232.000000,3232.000000,3232.000000,3232.000000,3232.000000,3232.000000,3232.000000,3232.000000,3232.000000
mean,4.422030,2875.824567,66.135769,0.642719,3.559642,13.483213,3.725014,1.498453,0.520287
std,2.504243,131.006274,30.854826,0.241845,5.228948,7.698980,0.896152,0.500075,0.286792
min,1.000000,2573.500000,13.250000,-0.025949,-21.832000,0.550000,0.200000,1.000000,0.010000
25%,2.000000,2791.000000,46.918750,0.492750,1.163750,8.346750,3.100000,1.000000,0.273000
50%,4.000000,2893.500000,65.721500,0.624437,3.500000,12.150000,3.551500,1.000000,0.526000
75%,6.000000,2980.000000,79.626250,0.812735,6.432500,16.453750,4.300000,2.000000,0.767250
max,9.000000,3122.500000,361.150000,1.480000,18.600000,84.400000,8.094000,2.000000,1.000000


In [3]:
test_well = data[data['Well Name'] == 'SHANKLE']
data = data[data['Well Name'] != 'SHANKLE']

In [4]:
features = ['GR', 'ILD_log10', 'DeltaPHI',
'PHIND','PE','NM_M', 'RELPOS']
feature_vectors = data[features]
facies_labels = data['Facies']

In [5]:
import seaborn as sns
sns.pairplot(feature_vectors[['GR', 'ILD_log10',
'DeltaPHI', 'PHIND','PE']])

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors)

In [7]:
from sklearn.cross_validation import train_test_split
X_train, X_cv, y_train, y_cv = \
train_test_split(scaled_features, facies_labels,
test_size=0.05, random_state=42)

In [8]:
from sklearn import svm
clf = svm.SVC(C=10, gamma=1)

In [9]:
clf.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
y_test = test_well['Facies']
well_features = test_well.drop(['Facies', 'Formation', 'Well Name', 'Depth'], axis=1)
X_test = scaler.transform(well_features)

In [11]:
y_pred = clf.predict(X_test)
test_well['Prediction'] = y_pred

In [12]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh',
'MS', 'WS', 'D','PS', 'BS']
print(classification_report(y_test, y_pred,
target_names=target_names))

             precision    recall  f1-score   support

         SS       0.30      0.08      0.12        89
       CSiS       0.36      0.72      0.48        89
       FSiS       0.62      0.54      0.58       117
       SiSh       0.25      0.29      0.27         7
         MS       0.17      0.11      0.13        19
         WS       0.66      0.54      0.59        71
          D       0.71      0.29      0.42        17
         PS       0.41      0.60      0.49        40
         BS       0.00      0.00      0.00         0

avg / total       0.47      0.46      0.43       449



C:\Users\jnewb\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1076: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
